Importing Libraries

In [3]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from google.colab import files
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

Upload the Data to Colab

In [4]:
print('\nUpload the spotify file')
File = files.upload()



Upload the spotify file


Saving data.csv to data (7).csv


Load the dataset

In [5]:
df = pd.read_csv("data.csv")

Check Data File

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169909 entries, 0 to 169908
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   acousticness      169909 non-null  float64
 1   artists           169909 non-null  object 
 2   danceability      169909 non-null  float64
 3   duration_ms       169909 non-null  int64  
 4   energy            169909 non-null  float64
 5   explicit          169909 non-null  int64  
 6   id                169909 non-null  object 
 7   instrumentalness  169909 non-null  float64
 8   key               169909 non-null  int64  
 9   liveness          169909 non-null  float64
 10  loudness          169909 non-null  float64
 11  mode              169909 non-null  int64  
 12  name              169909 non-null  object 
 13  popularity        169909 non-null  int64  
 14  release_date      169909 non-null  object 
 15  speechiness       169909 non-null  float64
 16  tempo             16

Feature scaling

In [7]:
col = ['id', 'name', 'artists', 'release_date', 'year','popularity']
df = df[df['year']>2003]
df.reset_index(drop=True, inplace=True)
#Numeric features that define the song
df1 = df.drop(columns=col)
scaler = MinMaxScaler()
df1_new = scaler.fit_transform(df1)
#Song Identifier like name, artist etc.
df["artists"]= df["artists"].str.replace("[", "")
df["artists"]= df["artists"].str.replace("]", "")
df["artists"]= df["artists"].str.replace("'", "")
df2 = df.loc[:, col]
#df.head(20)

In [8]:
df.head(20)
len(df)

33656

In [9]:
len(df1_new)

33656

Item - Item Filtering using Manhattan Distance

In [10]:
#content filter
class Spotify_Recommendation():
    def __init__(self, dataset):
        self.dataset = dataset
    def recommend(self, songs, amount=1):
        distance = []
        song = self.dataset[(self.dataset.name.str.lower() == songs.lower())].head(1).values[0]
        rec = self.dataset[self.dataset.name.str.lower() != songs.lower()]
        for songs in tqdm(rec.values):
            d = 0
            for col in np.arange(len(rec.columns)):
                if not col in [1, 6, 12, 14, 18]: #excluding all non numeric data
                    d = d + np.absolute(float(song[col]) - float(songs[col]))
            distance.append(d)
        rec['distance'] = distance
        rec = rec.sort_values('distance')
        columns = ['artists', 'name','release_date', 'year','id']
        return rec[columns][:amount]

In [11]:
h = 30000
model = Spotify_Recommendation(df.head(h))
m1 = model.recommend("Harleys In Hawaii", 10)     #Wake,Legends Are Made,Harleys In Hawaii,Waiting for the End,Berzerk,Hotline Bling,God's Plan
m1

100%|██████████| 29999/29999 [00:02<00:00, 13992.15it/s]


,artists,name,release_date,year,id
20882,Alan Jackson,Chasin' That Neon Rainbow,2010-11-19,2010,5Wp1Y8jscofORVp0FH0N9l
7900,LIZ,When I Rule the World,2015-07-17,2015,7nu0Lc0jJltztDxsGeoPiG
18688,Miranda Lambert,Kerosene,2005,2005,2yhhcHTfvLC0HzSajGYL0h
28877,Priestess,Lay Down,2006-06-13,2006,4ltXPf327ewFdmTPp5Skow
6099,Attila,About That Life,2013-06-25,2013,2kg6xSbkMXH36655I5iJMN
21825,"Roddy Ricch, A Boogie Wit da Hoodie",Tip Toe (feat. A Boogie Wit da Hoodie),2019-12-06,2019,6ztvsy3C6hPjVg9j4x1XKJ
20140,YoungBoy Never Broke Again,Self Control,2019-10-11,2019,4vSoyDfL0uZyUoWu5NNNsQ
10748,Three Days Grace,Goin' Down,2009-09-22,2009,5fMOlPVPDpeOqhSSX4cnfu
10032,"Roddy Ricch, A Boogie Wit da Hoodie",Tip Toe (feat. A Boogie Wit da Hoodie),2019-11-25,2019,5j1yOqWONR9T6l43AzJ6Es
26422,"Nate Dogg, J.PERIOD",So Fly,2015-04-14,2015,2fxef3Rm1GnrcyGjvWP8oJ


Item - Item Filtering using Cosine Similarity

In [12]:
class Spotify_Recommendation2():
    def __init__(self, df):
        self.df = df
    def recommendation(self, name,n):
        # Create a pandas series with song titles as indices and indices as series values
        h = 30000 
        indices = pd.Series(self.df.head(h).index, index=self.df['name'].head(h)).drop_duplicates()
        col = ['id', 'name', 'artists', 'release_date', 'year']
        #Numeric features that define the song
        df1 = self.df.drop(columns=col)
        scaler = MinMaxScaler()
        df1_new = scaler.fit_transform(df1)
        # Create cosine similarity matrix based on given matrix
        c = pd.DataFrame(df1_new)
        c = c.head(h)
        cosine = cosine_similarity(c)
        # Get song indices
        self.name = name
        index=indices[self.name]
        # Get list of songs for given songs
        score=list(enumerate(cosine[indices[self.name]]))
        # Sort the most similar songs
        similarity_score = sorted(score,key = lambda x:x[1],reverse = True)
        # Select the top-10 recommend songs
        similarity_score = similarity_score[1:n+1]
        top_songs_index = [i[0] for i in tqdm(similarity_score)]
        # Top 10 recommende songs
        top_songs=df.iloc[top_songs_index]
        columns = ['artists', 'name','release_date', 'year','id']
        return top_songs[columns]

In [13]:
model2 = Spotify_Recommendation2(df)

In [14]:
m2 = model2.recommendation("Harleys In Hawaii",10)
m2

100%|██████████| 10/10 [00:00<00:00, 75437.12it/s]


,artists,name,release_date,year,id
21384,"Shawn Mendes, Camila Cabello",I Know What You Did Last Summer,2015-04-14,2015,2GyA33q5rti5IxkMQemRDH
17304,Rihanna,Disturbia,2008-06-02,2008,2VOomzT6VavJOGBeySqaMc
8271,"Prince Royce, Maluma",El Clavo (feat. Maluma) - Remix,2018-05-11,2018,6bI00brFpeprqCDqKouEJS
17138,Aventura,Los Infieles,2006-12-19,2006,0HDHY6RSHHG2ZTE0cMT4GJ
24646,"Shakira, Rihanna",Can't Remember to Forget You (feat. Rihanna),2014-03-14,2014,2Sh4sAOfnSHEVKFyysxzat
21736,"Pedro Capó, Farruko",Calma - Remix,2018-10-05,2018,5iwz1NiezX7WWjnCgY5TH4
3321,"Gente De Zona, Becky G",Muchacha,2020-04-23,2020,2vI4SxtYqAQD5h1nSrqfKM
6510,BTS,Go Go,2018-08-24,2018,1ok4gxOv8cg5WLjWK6TQhD
6767,The Pussycat Dolls,React,2020-02-07,2020,0GWYApQBwErVPkyXYCTJjI
18176,Imagine Dragons,Believer,2017-06-23,2017,0pqnGHJpmpxLKifKRmU6WP


Find Common Songs recommended by both recommenders and check their rankings as per the recommenders

In [15]:
m1['Rank m1'] = range(1,len(m1)+1)
m2['Rank m2'] = range(1,len(m2)+1)
idx = m1.index.intersection(m2.index) 
print('Total Common Items are ',len(idx))
col = ['name', 'artists','year']
mf = df.loc[idx, col]
print (mf) if len(idx)>0 else print('No items are common')

Total Common Items are  0
No items are common


In [16]:
if len(idx)> 0:
    f1 = pd.merge(mf,m1,on='name')
    f1 = pd.merge(f1,m2,on='name')
    col = ['name','Rank m1','Rank m2']
    f2 = f1.loc[:,col]
    print(f2)
else:
    print('No items are common')

No items are common


Using K means to understand clusters of the songs

In [20]:
kmeans = KMeans(n_clusters=50)
df1_new = pd.DataFrame(df1_new)
d = df1_new.head(h)
features = kmeans.fit_predict(d)
df = df.head(h)
df['clusters'] = features
MinMaxScaler(df['clusters'])

MinMaxScaler(feature_range=0        10
1        21
2        29
3        45
4        41
         ..
29995    32
29996    44
29997    27
29998     4
29999     9
Name: clusters, Length: 30000, dtype: int32)

In [21]:
Final1 = pd.merge(m1,df,on='id')
Final2 = pd.merge(m2,df,on='id')
col = ['name_x','artists_x','clusters']
Final1 = Final1.loc[:,col]
Final2 = Final2.loc[:,col]

In [22]:
print(Final1)
print(Final2)

                                   name_x  \
0               Chasin' That Neon Rainbow   
1                   When I Rule the World   
2                                Kerosene   
3                                Lay Down   
4                         About That Life   
5  Tip Toe (feat. A Boogie Wit da Hoodie)   
6                            Self Control   
7                              Goin' Down   
8  Tip Toe (feat. A Boogie Wit da Hoodie)   
9                                  So Fly   

                             artists_x  clusters  
0                         Alan Jackson        44  
1                                  LIZ        19  
2                      Miranda Lambert        44  
3                            Priestess         3  
4                               Attila        34  
5  Roddy Ricch, A Boogie Wit da Hoodie        25  
6           YoungBoy Never Broke Again        16  
7                     Three Days Grace        22  
8  Roddy Ricch, A Boogie Wit da Hoodie       

Note - The program is run only for the first 35,000 rows so that the system does not take a lot of time and memory to compute the distances and similarities